## 使用字符RNN生成莎士比亚文本

在2015年的一篇著名博客文章 "https://karpathy.github.io/2015/05/21/rnn-effectiveness/" 中，Andrej Karpathy展示了如何训练循环神经网络来预测句子中的下一个字符。这个char-RNN可以用来生成小说文本，每次一个字符。

In [14]:
# tf.keras.utils.get_file()函数下载莎士比亚的所有作品
import tensorflow as tf

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [15]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [16]:
# 使用tf.keras.layers.TextVectorization层对此文本进行编码。我们将设置split="character"，基于字符进行编码，而不是默认的基于单词进行编码，并使用standardize="lower"将文本转换为小写（这将简化任务）：
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
encoded

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [17]:
# 每个字符现在都映射到一个整数，从2开始。TextVectorization层将值0保留为填充词元(token)，并将值1保留为未知字符。
# 现在不需要这两个词元，因此从字符ID数中减去2，并计算不同字符的总数和总字符数：
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

将这个非常长的序列转换成一个窗口数据集，然后使用它来训练一个序列到序列的循环神经网络。目标序列将类似于输入序列，但是会向“未来”移动一个时间步。例如，数据集中的样本可能是一个由表示文本“to be or not to b”（不包括最后一个“e”）的字符ID组成的序列，相应的目标序列是一个由表示文本“o be or not to be”（包括最后一个“e”，但不包括开头的“t”）的字符ID组成的序列

In [18]:
# 将字符ID序列转换成输入/目标窗口对的数据集
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)

    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

- 它将序列作为输入（即编码文本），并创建一个包含所需长度的所有窗口的数据集。
- 它将长度增加1，因为我们需要将下一个字符放在目标序列中。
- 然后，它对窗口进行乱序处理（可选），将它们分批，将它们拆分为输入/输出对，并激活预取功能。

长度为11的窗口和批量大小3。每个窗口的起始索引都显示在旁边：

![准备乱序窗口的数据集](./images/RNN/p7.png)

In [2]:
# 大约90%的文本进行训练，5%用于验证，5%用于测试
length = 100  # length决定循环神经网络能学习的最长模式
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000: 1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [13]:
for data in test_set.take(1):
    print(data)

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 2, 21,  5, ...,  2,  4,  8],
       [21,  5, 10, ...,  4,  8,  7],
       [ 5, 10,  2, ...,  8,  7,  9],
       ...,
       [10, 27,  6, ..., 10, 11,  7],
       [27,  6, 11, ..., 11,  7, 11],
       [ 6, 11,  4, ...,  7, 11, 22]], dtype=int64)>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[21,  5, 10, ...,  4,  8,  7],
       [ 5, 10,  2, ...,  8,  7,  9],
       [10,  2, 17, ...,  7,  9, 19],
       ...,
       [27,  6, 11, ..., 11,  7, 11],
       [ 6, 11,  4, ...,  7, 11, 22],
       [11,  4,  2, ..., 11, 22, 19]], dtype=int64)>)


### 构建和训练char-RNN模型

- 使用Embedding层作为第一层，以编码字符ID。Embedding层的输入维数是不同字符ID的数量，输出维数是可以调整的超参数——现在将其设置为16。Embedding层的输入将是形状为［批量大小，窗口长度］的二维张量，Embedding层的输出将是形状为［批量大小，窗口长度，嵌入大小］的三维张量。
- 为输出层使用Dense层：它必须具有39个单元(n_tokens)，因为文本中有39个不同的字符，希望在每个时间步输出每个可能字符的概率。每个时间步39个输出概率的总和应该为1，因此对Dense层的输出应用softmax激活函数。
- 最后，使用"sparse_categorical_crossentropy"损失和Nadam优化器编译此模型，并使用ModelCheckpoint回调函数在训练过程中保存最佳模型（以验证精度为标准）进行多个轮次的训练。

In [20]:
# 不用简单RNN,用带有长短记忆的GRU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True
)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

Epoch 1/10
  31247/Unknown - 1123s 36ms/step - loss: 1.4060 - accuracy: 0.5702INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 1144s 36ms/step - loss: 1.4060 - accuracy: 0.5702 - val_loss: 1.6281 - val_accuracy: 0.5289
Epoch 2/10
31247/31247 [==============================] - ETA: 0s - loss: 1.3079 - accuracy: 0.5929INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 1137s 36ms/step - loss: 1.3079 - accuracy: 0.5929 - val_loss: 1.5923 - val_accuracy: 0.5355
Epoch 3/10
31246/31247 [============================>.] - ETA: 0s - loss: 1.2904 - accuracy: 0.5967INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

31247/31247 [==============================] - 1454s 46ms/step - loss: 1.2663 - accuracy: 0.6016 - val_loss: 1.5715 - val_accuracy: 0.5421
Epoch 8/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2639 - accuracy: 0.6021INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 1374s 44ms/step - loss: 1.2639 - accuracy: 0.6021 - val_loss: 1.5679 - val_accuracy: 0.5437
Epoch 9/10
31247/31247 [==============================] - 1306s 42ms/step - loss: 1.2621 - accuracy: 0.6025 - val_loss: 1.5736 - val_accuracy: 0.5433
Epoch 10/10
31247/31247 [==============================] - ETA: 0s - loss: 1.2598 - accuracy: 0.6029INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31247/31247 [==============================] - 1306s 42ms/step - loss: 1.2598 - accuracy: 0.6029 - val_loss: 1.5723 - val_accuracy: 0.5445


In [21]:
# 训练后搭建最终模型
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2), # 不使用填充词元(0)  和 未知词元 (1)
    model
])

In [22]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # 选择概率最高的字母ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 1s 645ms/step


'e'

### 生成莎士比亚文本

要使用char-RNN模型生成新文本，可以将一些文本输入模型，让模型预测最有可能的下一个字母，将其添加到文本的末尾，然后将扩展后的文本输入模型来猜测下一个字母，以此类推。这叫作贪婪解码。但是在实践中，这往往导致相同的单词一遍又一遍地重复。

相反，可以使用TensorFlow的tf.random.categorical()函数随机采样下一个字符，采样概率等于估计概率。这将生成更多样化和有趣的文本。categorical()函数在给定类别对数概率(logits)的情况下，对随机类别指数进行采样。

In [23]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)


<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]], dtype=int64)>

为了更好地控制生成文本的多样性，可以将logits（对数概率）除以一个称为“温度”的数字，这个数字可以根据需求进行调整。“温度”接近零将更偏向于高概率字符，而较高“温度”则会使所有字符获得相同的概率。通常在生成相对严谨和精确的文本（例如数学公式）时，较低的“温度”更为适用，而在生成更多样化且有创意的文本时，则适合用较高的“温度”。

In [24]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [25]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [27]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))


1/1 [==============================] - 0s 31ms/step
To be or not to be the rest of the statute of the statute of his par


In [28]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 16ms/step
To be or not to be good and wrong; but come. though which the fine.



In [29]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 16ms/step
To be or not to bef ,mt'&ozfpady-$
wh!nse?pws3ert--vgerdjw?c-y-ewxnj


### 模拟生成莎士比亚文本的流程 生成名字

In [30]:
with open("datasets/dino.txt", "r") as f:
    dino_names = f.read()
    dino_names = dino_names.lower()

In [31]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([dino_names])
encoded = text_vec_layer([dino_names])[0]
encoded

<tf.Tensor: shape=(19909,), dtype=int64, numpy=array([ 2,  2, 15, ...,  4,  4, 12], dtype=int64)>

In [32]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 'a',
 's',
 'u',
 'o',
 'r',
 '\n',
 'n',
 'i',
 'e',
 't',
 'l',
 'p',
 'h',
 'c',
 'g',
 'd',
 'm',
 'y',
 'b',
 'k',
 'v',
 'x',
 'z',
 'j',
 'w',
 'f',
 'q']

In [33]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
n_tokens

27

In [46]:
end_of_name_encode = text_vec_layer(["\n"])[0,0].numpy() - 2
end_of_name_encode

5

In [110]:
# 把 Tensor 转成 numpy 数组
encoded_np = encoded.numpy()

# 存放 (X, Y) 对
names_X = []
names_Y = []

# 临时缓冲区
current_name = []

for token in encoded_np:
    current_name.append(token)
    if token == end_of_name_encode:
        if len(current_name) > 1:  # 至少两个字符才能形成 X/Y
            X = current_name[:-1]
            Y = current_name[1:]
            names_X.append(X)
            names_Y.append(Y)
        current_name = []


if len(current_name) > 1:
    X = current_name[:-1]
    Y = current_name[1:]
    names_X.append(X)
    names_Y.append(Y)
    current_name = []

# 转成 TensorFlow 张量
X_dataset = tf.ragged.constant(names_X)
Y_dataset = tf.ragged.constant(names_Y)

print("共提取名字数:", len(names_X))
print("X_dataset 形状:", X_dataset.shape)
print("Y_dataset 形状:", Y_dataset.shape)

Y_dataset

共提取名字数: 1536
X_dataset 形状: (1536, None)
Y_dataset 形状: (1536, None)


<tf.RaggedTensor [[0, 13, 12, 8, 6, 3, 1, 0, 2, 4, 2, 1, 5], [0, 4, 15, 3, 6, 17, 21, 5],
 [18, 15, 0, 10, 10, 0, 12, 1, 0, 2, 4, 2, 1, 5], ...,
 [2, 3, 17, 2, 6, 10, 3, 6, 14, 5], [2, 11, 0, 17, 1, 0, 2, 4, 2, 1, 5],
 [2, 2, 10]]>

In [111]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(X_dataset, Y_dataset, batch_size=1, epochs=10)

Epoch 1/10
1536/1536 [==============================] - 7s 3ms/step - loss: 2.0521 - accuracy: 0.3971
Epoch 2/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.7431 - accuracy: 0.4802
Epoch 3/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.6397 - accuracy: 0.5152
Epoch 4/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.5672 - accuracy: 0.5377
Epoch 5/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.5101 - accuracy: 0.5524
Epoch 6/10
1536/1536 [==============================] - 5s 4ms/step - loss: 1.4546 - accuracy: 0.5663
Epoch 7/10
1536/1536 [==============================] - 5s 4ms/step - loss: 1.3978 - accuracy: 0.5814
Epoch 8/10
1536/1536 [==============================] - 5s 4ms/step - loss: 1.3432 - accuracy: 0.5918
Epoch 9/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.2854 - accuracy: 0.6109
Epoch 10/10
1536/1536 [==============================] - 5s 3ms/step - loss: 1.226

In [112]:
dino_name_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])

In [113]:
def my_next_char(model, text, temperature=1):
    y_proba = model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [114]:
def my_extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        char_gen_next = my_next_char(dino_name_model, text, temperature)
        if char_gen_next == "\n":
            break
        text += my_next_char(dino_name_model, text, temperature)
    return text

In [119]:
my_extend_text("man")

1/1 [==============================] - 0s 22ms/step


'mansodon'